# Delta Lake デモンストレーション

このノートブックでは、Delta Lakeの主要な機能を実際のコードを通じて学習します。

## 学習内容
1. **Time Travel**: データの履歴管理と過去の状態への復旧
2. **VACUUM**: ストレージ容量の最適化と古いファイルの削除
3. **Liquid Clustering**: クエリパフォーマンスの向上

## 前提条件
- Unity Catalogが有効化されている環境

## 1. Time Travel（タイムトラベル）

Delta Lakeの**Time Travel**機能は、テーブルの過去の状態を参照・復旧できる強力な機能です。

### 主な用途
- **誤操作からの復旧**: 間違ってデータを削除・更新した場合の復元
- **データの監査**: 特定時点でのデータ状態の確認
- **A/Bテスト**: 異なるバージョンのデータでの比較分析

### 仕組み
- Delta Lakeは各操作をバージョンとして管理
- 過去のデータファイルは自動的に保持される
- `VERSION AS OF`や`TIMESTAMP AS OF`で過去の状態にアクセス可能

In [0]:
-- Unity Catalogのパーソナル領域を使用
-- 各ユーザーは自分のスキーマ名に変更してください
USE CATALOG users;
USE SCHEMA yukiteru_koide;  -- ← ここを自分のユーザー名に変更


In [0]:
### ステップ1: デモ用テーブルの作成と初期データ投入

Delta Lakeテーブルを作成し、初期データを投入します。この操作により最初のバージョン（バージョン0）が作成されます。


In [ ]:
-- Delta Lakeテーブルを作成（USING DELTAを指定）
CREATE OR REPLACE TABLE deltalake (
  id       INT,           -- 主キー
  amount   DECIMAL(10,2), -- 金額
  country  STRING,        -- 国コード
  ts       TIMESTAMP      -- タイムスタンプ
) USING DELTA;

-- 初期データを3件投入（これがバージョン0になる）
INSERT INTO deltalake VALUES
  (1, 100.00, 'JP', current_timestamp()),
  (2, 200.00, 'US', current_timestamp()),
  (3, 150.00, 'JP', current_timestamp());

-- 投入されたデータを確認
SELECT * FROM deltalake ORDER BY id;


In [0]:
### ステップ2: 誤操作のシミュレーション

**⚠️ 注意**: これは意図的な誤操作です！

日本（JP）のデータを間違って全削除してしまう状況を再現します。実際の運用では絶対に避けるべき操作ですが、復旧デモのために実行します。



In [ ]:
-- 日本（JP）のデータを誤って削除
DELETE FROM deltalake WHERE country = 'JP';

-- 削除後の状態を確認（JPのデータが消えていることを確認）
SELECT * FROM deltalake ORDER BY id;


In [0]:
### ステップ3: テーブル履歴の確認

`DESCRIBE HISTORY`でテーブルの操作履歴を確認します。各バージョンの操作内容、実行時刻、ファイル数などが表示されます。

**出力項目の説明:**
- **version**: バージョン番号（0から始まる連番）
- **timestamp**: 操作実行時刻
- **operation**: 実行された操作（CREATE TABLE, INSERT, DELETE等）
- **operationParameters**: 操作の詳細パラメータ
- **readVersion**: 読み取り時のバージョン
- **isBlindAppend**: データ追加のみかどうか



In [ ]:
-- テーブルの操作履歴を確認
DESCRIBE HISTORY deltalake;


In [0]:
### ステップ4: 過去バージョンのデータ参照

`VERSION AS OF`句を使用して特定バージョンのデータを参照します。削除前の状態（通常はバージョン0）を確認してみましょう。

**補足**: タイムスタンプでの参照も可能です
```sql
SELECT * FROM deltalake TIMESTAMP AS OF '2024-01-01 12:00:00' ORDER BY id;
```

※ 実際のバージョン番号は上記の`DESCRIBE HISTORY`の結果に合わせて調整してください



In [ ]:
-- 削除前の状態を参照（バージョン0のデータ）
SELECT * FROM deltalake VERSION AS OF 0 ORDER BY id;


In [0]:
### ステップ5: テーブルの復旧（RESTORE）

`RESTORE TABLE`コマンドで指定バージョンの状態にテーブル全体を復旧します。これにより、誤って削除されたデータを一瞬で復元できます。

**RESTOREの特徴:**
- テーブル全体が指定バージョンの状態に戻る
- 新しいバージョンとして記録される（履歴は残る）
- タイムスタンプでの復旧も可能: `RESTORE TABLE deltalake TO TIMESTAMP AS OF '2024-01-01'`

※ 上記の`DESCRIBE HISTORY`で確認した削除前のバージョン番号を指定してください



In [ ]:
-- テーブルを削除前の状態に復旧
RESTORE TABLE deltalake TO VERSION AS OF 0;


In [0]:
### ステップ6: 復旧結果の確認

復旧後のデータを確認します。削除されていた日本（JP）のデータが復元されていることを確認しましょう。

復旧操作も履歴に記録されているため、必要に応じて`DESCRIBE HISTORY deltalake`で確認できます。

In [ ]:
-- 復旧後のデータを確認
SELECT * FROM deltalake ORDER BY id;

-- 復旧操作も履歴に記録されているかを確認
-- DESCRIBE HISTORY deltalake;


## 2. VACUUM（ストレージ最適化）

Delta Lakeの**VACUUM**機能は、不要になった古いデータファイルを物理削除してストレージ容量を最適化します。

### VACUUMの目的
- **ストレージコスト削減**: 古いバージョンのファイルを削除して容量を節約
- **パフォーマンス向上**: ファイル数の削減によりクエリ性能を改善
- **データ管理**: 長期間不要なファイルの自動削除

### 重要な注意点
- **Time Travel機能への影響**: VACUUMで削除されたバージョンは参照不可能になる
- **保持期間の設定**: デフォルトは7日間、業務要件に応じて調整が必要
- **DRY RUN**: 実際の削除前に削除対象ファイルを確認することを強く推奨

In [0]:
### ステップ1: VACUUM実行前の履歴確認

現在のテーブル履歴を確認します（VACUUM前の状態）。複数のバージョンが存在することを確認しましょう。

In [ ]:
-- 現在のテーブル履歴を確認
DESCRIBE HISTORY users.yukiteru_koide.deltalake;


In [0]:
-- ### ステップ2: VACUUM前のTime Travel動作確認

-- VACUUM実行前は古いバージョンにアクセス可能であることを確認
-- ※ 実際に存在するバージョン番号に調整してください
SELECT * FROM users.yukiteru_koide.deltalake VERSION AS OF 1;

### ステップ2: VACUUM前のTime Travel動作確認

VACUUM実行前は古いバージョンにアクセス可能であることを確認します。

※ 実際に存在するバージョン番号に調整してください


In [ ]:
-- 古いバージョンにアクセス可能であることを確認
SELECT * FROM users.yukiteru_koide.deltalake VERSION AS OF 1;


### ステップ3: VACUUM DRY RUN（削除対象ファイルの事前確認）

`DRY RUN`オプションで削除対象ファイルを確認します。実際にはファイルを削除せず、削除対象のファイルパスのみを表示します。

**重要**: 本番環境では必ずDRY RUNで確認してから実行することを推奨します。

**DRY RUNの出力:**
- 削除対象のParquetファイルのパスが表示される
- ファイル数とサイズを事前に把握できる


In [0]:
-- 削除対象ファイルを事前に確認
VACUUM users.yukiteru_koide.deltalake DRY RUN;

### ステップ4: ファイル保持期間の変更（デモ用）

**⚠️ 注意**: これはデモ用の設定です！

通常のデフォルト保持期間は7日間（168時間）ですが、ここでは即座にVACUUMの効果を確認するため1時間に短縮します。本番環境では業務要件に応じて適切な期間を設定してください。

**保持期間の考慮事項:**
- **短すぎる**: Time Travelの利便性が低下
- **長すぎる**: ストレージコストが増大
- **推奨**: 業務の復旧要件に基づいて設定（通常1-30日）


In [0]:
-- デモ用に保持期間を1時間に設定
ALTER TABLE users.yukiteru_koide.deltalake
  SET TBLPROPERTIES ('delta.deletedFileRetentionDuration' = '1 hours');

### ステップ5: VACUUM実行

古いファイルを物理削除してストレージを最適化します。保持期間（1時間）を超えた古いバージョンのファイルが削除されます。

**VACUUM実行時の動作:**
- 保持期間を超えたParquetファイルを物理削除
- メタデータ（_delta_log）は保持される
- 削除されたファイル数とサイズが出力される


In [0]:

-- 古いファイルを物理削除
VACUUM users.yukiteru_koide.deltalake;

In [0]:
-- ### ステップ6: VACUUM後の履歴確認

-- VACUUM実行後もテーブル履歴は残っていることを確認
-- メタデータは保持されるため、履歴情報は参照可能
DESCRIBE HISTORY users.yukiteru_koide.deltalake;

In [0]:
-- ### ステップ7: VACUUM後のTime Travel動作確認

-- VACUUMで物理削除されたバージョンへのアクセスを試行
-- ファイルが削除されている場合はエラーが発生する
-- ※ 削除されたバージョン番号を指定してください
SELECT * FROM users.yukiteru_koide.deltalake VERSION AS OF 1;

-- 【期待される結果】
-- ・ファイルが削除済み: "FileNotFoundException" などのエラー
-- ・ファイルが残存: 正常にデータが表示される
-- ・最新バージョンは常にアクセス可能

### ステップ1: クラスタリング設定前の状態確認

`DESCRIBE EXTENDED`でテーブルの詳細情報を確認し、クラスタリング設定前の状態を記録します。

**確認ポイント:**
- **Table Properties**: 現在のテーブル設定
- **Clustering Information**: クラスタリング設定の有無
- **Statistics**: ファイル数、データサイズなどの統計情報


In [0]:
-- ### ステップ8: 設定の復元（推奨）

-- デモ用に変更した保持期間設定をデフォルト（7日間）に戻す
-- 本番環境では適切な保持期間を維持することが重要
ALTER TABLE users.yukiteru_koide.deltalake
  UNSET TBLPROPERTIES ('delta.deletedFileRetentionDuration');

-- 【設定復元の重要性】
-- ・デモ用の短い保持期間は本番環境では危険
-- ・デフォルト設定（7日間）は多くの用途で適切
-- ・組織のデータ保持ポリシーに従って設定を調整

## 3. Liquid Clustering（パフォーマンス最適化）

Delta Lakeの**Liquid Clustering**は、データの物理的な配置を最適化してクエリパフォーマンスを向上させる機能です。

### Liquid Clusteringの特徴
- **自動最適化**: 指定したカラムに基づいてデータを自動的にクラスタリング
- **動的調整**: データの追加・更新に応じて最適な配置を維持
- **従来手法の改善**: パーティショニングの制約を解決する新しいアプローチ

### パフォーマンス向上の仕組み
- **データスキップ**: 不要なファイルの読み取りを回避
- **I/O削減**: 関連データを同じファイルに配置して読み取り効率を向上
- **Z-Order最適化**: 多次元データの局所性を改善

### 適用場面
- **フィルタリングが多い**: WHERE句で特定カラムを頻繁に使用
- **結合処理**: JOIN操作でのパフォーマンス向上
- **分析クエリ**: 集計処理の高速化

In [0]:
-- ### ステップ1: クラスタリング設定前の状態確認

-- DESCRIBE EXTENDEDでテーブルの詳細情報を確認
-- クラスタリング設定前の状態を記録
DESCRIBE EXTENDED users.yukiteru_koide.deltalake;

-- 【確認ポイント】
-- ・Table Properties: 現在のテーブル設定
-- ・Clustering Information: クラスタリング設定の有無
-- ・Statistics: ファイル数、データサイズなどの統計情報

In [0]:
-- ### ステップ2: Liquid Clusteringの有効化

-- ALTER TABLE文でクラスタリングカラムを指定
-- ここでは'country'カラムでクラスタリングを設定
-- 国別でデータをグループ化することで、国別フィルタリングが高速化される
ALTER TABLE users.yukiteru_koide.deltalake
CLUSTER BY (country);

-- 【クラスタリングカラムの選択指針】
-- ・WHERE句で頻繁に使用されるカラム
-- ・JOIN条件で使用されるカラム
-- ・GROUP BYで使用されるカラム
-- ・カーディナリティが適度なカラム（極端に多い/少ないは避ける）

In [0]:
-- ### ステップ3: クラスタリング設定後の状態確認

-- クラスタリング設定後のテーブル情報を確認
-- Clustering Informationセクションに設定が反映されているかを確認
DESCRIBE EXTENDED users.yukiteru_koide.deltalake;

-- 【設定後の変化】
-- ・Clustering Information: "country"が表示される
-- ・Table Properties: クラスタリング関連のプロパティが追加
-- ・今後のデータ挿入・更新時に自動的にクラスタリングが適用される

-- 【パフォーマンス効果の確認方法】
-- ・EXPLAIN文でクエリ実行計画を確認
-- ・実際のクエリ実行時間を測定
-- ・ファイルスキップ統計を確認

## まとめ

このノートブックでは、Delta Lakeの3つの主要機能を実際のコードで体験しました。

### 学習した内容
1. **Time Travel**: データの履歴管理と誤操作からの復旧
   - バージョン管理による過去データの参照
   - RESTORE機能による一括復旧
   - 運用における重要性

2. **VACUUM**: ストレージ最適化とコスト管理
   - 古いファイルの物理削除
   - 保持期間の設定と管理
   - Time Travelとのバランス

3. **Liquid Clustering**: クエリパフォーマンスの向上
   - データの物理配置最適化
   - 自動クラスタリング機能
   - 適切なカラム選択の重要性

### 本番環境での注意点
- **Time Travel**: 適切な保持期間の設定
- **VACUUM**: 定期実行とDRY RUNの活用
- **Clustering**: ワークロードに応じたカラム選択

### 次のステップ
- より大きなデータセットでのパフォーマンス測定
- 複数カラムでのクラスタリング設定
- Change Data Feed（CDF）などの他のDelta Lake機能の探索
